### General Settings

In [ ]:
QUOTES_FILE = "/content/drive/MyDrive/modding/l4d2-mod/makeup-dub/quotes_makeup_update.json" #@param {type:"string"}
OUTPUT_DIR = "/content/drive/MyDrive/modding/l4d2-mod/makeup-dub/update/producer" #@param {type:"string"}
SHOW_PROGRESS = True #@param {type:"boolean"}
SOURCE_CHARACTER = "hifumi" #@param {type:"string"}
TARGET_CHARACTER = "producer" #@param {type:"string"}
QUOTE_LANG = "japanese" #@param {type:"string"}
IMPORT_MODEL = True #@param {type:"boolean"}
EXTRA_MODEL = "/content/drive/MyDrive/vits-finetune/finished-models/hifumi/checkpoints/G_4000.pth" #@param {type:"string"}
EXTRA_MODEL_CONFIG = "/content/drive/MyDrive/vits-finetune/finished-models/hifumi/checkpoints/config.json" #@param {type:"string"}

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!nvidia-smi

Sat Feb 25 10:31:23 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   59C    P0    28W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

### Install requirements

In [ ]:
!apt install git-lfs
!git lfs install
!git clone https://huggingface.co/spaces/sayashi/vits-models

Reading package lists... Done
Building dependency tree       
Reading state information... Done
git-lfs is already the newest version (2.9.2-1).
0 upgraded, 0 newly installed, 0 to remove and 19 not upgraded.
Error: Failed to call git rev-parse --git-dir: exit status 128 
Git LFS initialized.
Cloning into 'vits-models'...
remote: Enumerating objects: 318, done.
remote: Counting objects: 100% (318/318), done.
remote: Compressing objects: 100% (282/282), done.
remote: Total 318 (delta 99), reused 0 (delta 0), pack-reused 0
Receiving objects: 100% (318/318), 7.22 MiB | 14.63 MiB/s, done.
Resolving deltas: 100% (99/99), done.
Filtering content: 100% (26/26), 3.53 GiB | 90.35 MiB/s, done.


In [ ]:
%cd vits-models

/content/vits-models


In [ ]:
!pip install -r requirements.txt

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.6/90.6 KB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.9/235.9 KB 18.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 64.5 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.2/14.2 MB 93.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 85.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.5/50.5 KB 7.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.9/56.9 KB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.0/107.0 KB 10.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
#@title import extra model
if IMPORT_MODEL:
  import shutil, os
  new_model_path = "pretrained_models/" + SOURCE_CHARACTER
  if not os.path.exists(new_model_path):
    os.mkdir(new_model_path)
  shutil.copy(EXTRA_MODEL, new_model_path + "/" + SOURCE_CHARACTER + ".pth")

### Generate voice files

In [ ]:
#@title Define infer function
%matplotlib inline
import matplotlib.pyplot as plt
import IPython.display as ipd

import os
import json
import math
import torch


import commons
import utils
from models import SynthesizerTrn
from text.symbols import symbols
from text import text_to_sequence

from scipy.io.wavfile import write


def get_text(text, hps):
    text_norm, clean_text = text_to_sequence(text, hps.symbols, hps.data.text_cleaners)
    if hps.data.add_blank:
        text_norm = commons.intersperse(text_norm, 0)
    text_norm = torch.LongTensor(text_norm)
    return text_norm

In [ ]:
#@title Load model
if IMPORT_MODEL:
  config_path = EXTRA_MODEL_CONFIG
else:
  config_path = "config/config.json"
#model_path = "/pretrained_models/" #@param {type:"string"}
model_path = os.path.join("pretrained_models", SOURCE_CHARACTER, SOURCE_CHARACTER + ".pth")
hps = utils.get_hparams_from_file(config_path)
net_g = SynthesizerTrn(
    len(hps.symbols),
    hps.data.filter_length // 2 + 1,
    hps.train.segment_size // hps.data.hop_length,
    n_speakers=hps.data.n_speakers,
    **hps.model).cuda()
model = net_g.eval()
model = utils.load_checkpoint(model_path, net_g, None)

In [ ]:
#@title Voice settings
speaker_id = 10 #@param {type:"number"}
noise_scale=0.6 #@param {type:"number"}
noise_scale_w=0.668 #@param {type:"number"}
length_scale=1.0 #@param {type:"number"}

def speak(text):
  if not IMPORT_MODEL:
    text = f"[JA]{text}[JA]"
  stn_tst = get_text(text, hps)
  with torch.no_grad():
      x_tst = stn_tst.cuda().unsqueeze(0)
      x_tst_lengths = torch.LongTensor([stn_tst.size(0)]).cuda()
      sid = torch.LongTensor([speaker_id]).cuda()
      audio = net_g.infer(x_tst, x_tst_lengths, sid=sid, noise_scale=noise_scale, noise_scale_w=noise_scale_w, length_scale=length_scale)[0][0,0].data.cpu().float().numpy()

  return hps.data.sampling_rate, audio

In [ ]:
#@title  Start generating quotes
# load quotes file
with open(QUOTES_FILE, "r", encoding="utf-8") as f:
  quotes = json.load(f)
  
# create output folder
if not os.path.exists(OUTPUT_DIR):
  os.mkdir(OUTPUT_DIR)

# traverse quotes 
print("Generating audios from quotes...")
char_quotes = quotes[TARGET_CHARACTER]
for filename in char_quotes:
  text = char_quotes[filename]["quote"][QUOTE_LANG]
  # generate audio
  if text is not None:
    try:
      rate, audio = speak(text)
      # save file
      if audio is not None:
        write(os.path.join(OUTPUT_DIR, filename), rate, audio)
        print("Generated " + os.path.join(OUTPUT_DIR, filename))
        print(text)
      else:
        print("Couldn't generate audio " + filename)
    except:
      print("There was an error while generating " + filename)
  else:
    print("Couldn't find a quote for " + filename)
  
print("All quotes were generated!")

Generating audios from quotes...
Generated /content/drive/MyDrive/modding/l4d2-mod/makeup-dub/update/producer/doubledeathresponse01
OK、OK、あー、神様! これは間違いだった!
Generated /content/drive/MyDrive/modding/l4d2-mod/makeup-dub/update/producer/friendlyfire25
誰を撃ってんのよ
Generated /content/drive/MyDrive/modding/l4d2-mod/makeup-dub/update/producer/friendlyfirec1nick01
そこのスーツ! 私を撃たないで!
Generated /content/drive/MyDrive/modding/l4d2-mod/makeup-dub/update/producer/gettingrevived01
ドクター、はっきり言って下さい。私は助かるんですか?
Generated /content/drive/MyDrive/modding/l4d2-mod/makeup-dub/update/producer/grabbedbysmoker03a
だめ、だめ
Generated /content/drive/MyDrive/modding/l4d2-mod/makeup-dub/update/producer/grabbedbysmoker04
いや!! いやー!!!!
Generated /content/drive/MyDrive/modding/l4d2-mod/makeup-dub/update/producer/heardhulk01
しまった、タンクだ!
Generated /content/drive/MyDrive/modding/l4d2-mod/makeup-dub/update/producer/help03
誰か助けて!
Generated /content/drive/MyDrive/modding/l4d2-mod/makeup-dub/update/producer/ledgehangslip01
きゃぁ
Generated